In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [0]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('scraped data.csv')
print(df.shape)

#remove null values

df.dropna(inplace=True)
df.isnull().sum()
print(df.columns)

(159571, 8)
Index(['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate'],
      dtype='object')


In [5]:
df.head(10)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
5,00025465d4725e87,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
7,00031b1e95af7921,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0
8,00037261f536c51d,Sorry if the word 'nonsense' was offensive to ...,0,0,0,0,0,0
9,00040093b2687caa,alignment on this subject and which are contra...,0,0,0,0,0,0


In [6]:
#check GPU Availability
import tensorflow as tf
tf.test.is_gpu_available()

True

In [0]:
df.describe()

classes = ['toxic','severe_toxic','obscene','threat','insult','identity_hate']
targets = df[classes].values

#perform feature engineering using GloVe Embeddings

embedding_file_path = 'drive/My Drive/glove.6B.100d.txt' #GloVe embedding file. Almost 350MB

In [8]:
targets[:2]

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [0]:
from tensorflow.keras.models import Model

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.layers import Input, Embedding, GRU, Dense, Activation, Dropout, CuDNNGRU
from tensorflow.keras.layers import RNN, concatenate
from tensorflow.keras.layers import Bidirectional, GlobalAveragePooling1D, GlobalMaxPool1D

from tensorflow.keras import initializers, optimizers

from tensorflow.keras.callbacks import ModelCheckpoint

In [0]:
features = df['comment_text'].fillna(' ').str.lower()

In [0]:
max_features, embed_size, maxlen = 100000, 100, 150
tokenizer = Tokenizer(max_features)

tokenizer.fit_on_texts(list(features))
sentences = tokenizer.texts_to_sequences(features)

In [12]:
sentences[0]

[688,
 75,
 1,
 126,
 130,
 177,
 29,
 672,
 4511,
 12052,
 1116,
 86,
 331,
 51,
 2278,
 11448,
 50,
 6864,
 15,
 60,
 2756,
 148,
 7,
 2937,
 34,
 117,
 1221,
 15190,
 2825,
 4,
 45,
 59,
 244,
 1,
 365,
 31,
 1,
 38,
 27,
 143,
 73,
 3462,
 89,
 3085,
 4583,
 2273,
 985]

In [0]:
padded_features = pad_sequences(sentences, maxlen=maxlen)

In [0]:
def get_coefs(word, *arr): 
    return word, np.asarray(arr, dtype='float32')

#costly operations, do it at your own risk
embeddings_index = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in open(embedding_file_path))

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.zeros((nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [15]:
#build the Deep Learning model

orig_input = Input(shape=(maxlen,)) #series
embed_layer = Embedding(max_features, embed_size, weights = [embedding_matrix], trainable=False)(orig_input)
bi_layer = Bidirectional(CuDNNGRU(128, return_sequences=True))(embed_layer)
avg_pool = GlobalAveragePooling1D()(bi_layer)
max_pool = GlobalMaxPool1D()(bi_layer)

conc_layer = concatenate([avg_pool, max_pool])
prediction_layer = Dense(6, activation='sigmoid')(conc_layer)

model = Model(inputs=orig_input, outputs=prediction_layer)
model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer=optimizers.Adam(clipnorm=1.))

print(model.summary())


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 150)]        0                                            
_________________________________________

In [0]:
saved_model_file = 'best_acc_toxic_weights.hdf5'
ckpt = ModelCheckpoint(saved_model_file, monitor='val_acc', save_best_only=True, save_weights_only=True, mode='max')

In [17]:
bs, ep = 32, 5

model.fit(padded_features, targets, validation_split=0.3, epochs=ep, batch_size=bs, callbacks=[ckpt])

Train on 111699 samples, validate on 47872 samples
Epoch 1/5
111699/111699 [==============================] - 70s 625us/sample - loss: 0.0597 - acc: 0.9790 - val_loss: 0.0499 - val_acc: 0.9817
Epoch 2/5
111699/111699 [==============================] - 58s 522us/sample - loss: 0.0463 - acc: 0.9826 - val_loss: 0.0480 - val_acc: 0.9822
Epoch 3/5
111699/111699 [==============================] - 58s 522us/sample - loss: 0.0421 - acc: 0.9841 - val_loss: 0.0466 - val_acc: 0.9826
Epoch 4/5
111699/111699 [==============================] - 58s 517us/sample - loss: 0.0384 - acc: 0.9853 - val_loss: 0.0473 - val_acc: 0.9827
Epoch 5/5
111699/111699 [==============================] - 58s 518us/sample - loss: 0.0346 - acc: 0.9867 - val_loss: 0.0477 - val_acc: 0.9824


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pickle

with open('tokenizer.pickle','wb') as f:
    pickle.dump(tokenizer, f, protocol=pickle.HIGHEST_PROTOCOL)

#save tokenizer to fit on texts during testing.

In [0]:
#save embedding matrix, model
with open('embedding_matrix.pickle', 'wb') as f:
    pickle.dump(embedding_matrix,f,protocol=pickle.HIGHEST_PROTOCOL)

model.save('Toxic_Comments_Model.model')

In [0]:
#call a testing / prediction line.

def get_coefs(word, *arr): 
    return word, np.asarray(arr, dtype='float32')

def test_prediction(sent, tokenizer='tokenizer.pickle', embedding_matrix='embedding_matrix.pickle', model='Toxic_Comments_Model.model'):

    
    if len(sent) == 0:
        print("No input entered - please enter a valid input")
        return
        
    import numpy as np
    import nltk
    import os

    import pickle

    from tensorflow.keras.models import Model
    from tensorflow.keras.optimizers import Adam
    from tensorflow.keras.models import load_model

    #do all the preprocessing you did earlier

    max_features, embed_size, maxlen = 100000, 100, 150

    tokenizer_file = open(tokenizer,'rb')
    tokenizer = pickle.load(tokenizer_file)
    tokenizer_file.close()

    features = sent.lower()

    sentences = tokenizer.texts_to_sequences([features])

    padded_features = pad_sequences(sentences, maxlen=maxlen) #features

    #run model.predict()

    model = load_model(model) #load the trained model
    label_list = model.predict(padded_features)
    print(label_list)

    predict_labels = []
    for item in label_list[0]:
        predict_labels.append(int(round(item)))
    
    classes = ["Toxic",", Severely Toxic",", Obscene",", a Threat",", an Insult",", Identity Hate"]

    print("The sentence : '{}', is classified as : \n".format(sent))
    for i in range(len(classes)):
        if predict_labels[i] == 1:
            print(classes[i],end=" ")

    return predict_labels, label_list[0]
    

In [50]:
proba, labels = test_prediction('I am hungry, you bitch. My hatred for you knows no bounds')

[[0.98371685 0.02448905 0.9282901  0.00607422 0.6033909  0.01199554]]
The sentence : 'I am hungry, you bitch. My hatred for you knows no bounds', is classified as : 

Toxic , Obscene , an Insult 